In [ ]:
#imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
#dados
train=pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test=pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [ ]:
#tratando os dados
#separando por titulo
for df in [train, test]:
    df["Title"] = df["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip() if ',' in x and '.' in x.split(',')[1] else '')
    df["Title"] = df["Title"].replace(["Don", "Rev", "Dr", "Major", "Lady", "Sir", "Col", "Capt", "the Countess", "Jonkheer"], "Rare")
    df["Title"] = df["Title"].replace(["Mlle", "Ms"], "Miss")
    df["Title"] = df["Title"].replace(["Mme"], "Mrs")

In [ ]:
target='Survived'
x_train,x_test,y_train,y_test=train_test_split(train.drop(target,axis=1),train[target],test_size=.3,random_state=0)
df_train=pd.concat([x_train,y_train],axis=1)
df_test=pd.concat([x_test,y_test],axis=1)
for i in [df_train,df_test,test]:
    i.drop(['Name','Ticket'],axis=1,inplace=True)

In [ ]:
categorical_features=[i for i in df_train.columns if df_train[i].dtype=='O']
numerical_features=[i for i in df_train.columns if df_train[i].dtype!='O']
print('categorical_features :\n',categorical_features,'\nnumerical_features :\n',numerical_features)

categorical_features :
 ['Sex', 'Cabin', 'Embarked', 'Title'] 
numerical_features :
 ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [ ]:
#irq
for feature in numerical_features:
    if 0 in df_train[feature].unique():
        pass
    else:
        q1=df_train[feature].quantile(0.25)
        q3=df_train[feature].quantile(0.75)
        iqr=q3-q1
        lower_limit=q1-1.5*iqr
        upper_limit=q3+1.5*iqr
        df_train=df_train[(df_train[feature]<upper_limit) & (df_train[feature]>lower_limit)]

In [ ]:
#tratando dados nulos
#preenchendo valores ausentes em 'Age' com a mediana
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

#preenchendo valores ausentes em 'Embarked' com o valor mais frequente
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

#preenchendo valores ausentes em 'Fare' com a mediana
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

for df in [df_train, df_test, test]:
    df['Cabin'].fillna('Unknown', inplace=True)
    df['Cabin'] = df['Cabin'].astype(str)

print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(test.isnull().sum())

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64


In [ ]:
print(train.shape)
print(test.shape)

(891, 13)
(418, 10)


In [ ]:
# Preparando os dados para o K-Means
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
X = df_train[features]
X_test = test[features]

In [ ]:
# Criando e treinando o modelo K-Means
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2, random_state=0)

In [ ]:
# Fazendo as predições
df_train['Survived'] = kmeans.predict(X)
test['Survived'] = kmeans.predict(X_test)

In [ ]:
result = test[['PassengerId', 'Survived']]
result.to_csv('titanicKmeans.csv', index=False)

In [ ]:
y_pred_train = kmeans.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
print("Acurácia no conjunto de treino:", accuracy_train)

precision_train = precision_score(y_train, y_pred_train)
print("Precisão no conjunto de treino:", precision_train)

tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
specificity_train = tn / (tn + fp)
print("Especificidade no conjunto de treino:", specificity_train)

recall_train = recall_score(y_train, y_pred_train)
print("Recall no conjunto de treino:", recall_train)

f1_train = f1_score(y_train, y_pred_train)
print("F1-score no conjunto de treino:", f1_train)


Acurácia no conjunto de treino: 1.0
Precisão no conjunto de treino: 1.0
Especificidade no conjunto de treino: 1.0
Recall no conjunto de treino: 1.0
F1-score no conjunto de treino: 1.0
